# Custom Activation Function

This example shows how to use a custom activation function in binet. We will implement this function both on the CPU and on the GPU. If you just want to quickly try something out, you can do without the GPU part.

Our activation function will be $h = max(0, \log(x+1))$. This might not make much sense as an activation function, but serves as a quick example as it's easy to implement.

In [1]:
from binet import *
from binet import layers

In [2]:
from pycuda.elementwise import ElementwiseKernel

__cuda_actfun =  ElementwiseKernel('float* x, float* o', 
                                   'o[i] = x[i] > 0 ? logf(x[i]+1.0f) : 0', 'actfun_eltw')
__cuda_actfun_delta = ElementwiseKernel('float* d, float* a, float* x', 
                                        'd[i] *= x[i] > 0 ? 1.0f / (x[i]+1.0f) : 0;', 'dactfun_eltw')

def my_activation(x, out=None, stream=None):
    if out is None:
        out = op.empty_like(x)
    if isinstance(x, op.gpuarray.GPUArray):
        __cuda_actfun(x, out, stream=stream)
    else:
        out[:] = np.where(x > 0, np.log(x+1), 0)
    return out  
    

def my_activation_delta(D, A, X, stream=None):
    """ Calculates D *= (a > 0)"""
    if isinstance(D, op.gpuarray.GPUArray):
        __cuda_actfun_delta(D, A, X, stream=stream)
    else:
        D *= np.where(X > 0, 1.0 / (X+1), 0)
    return D


# Register the new activation function
from binet import layers
layers.BasicLayer.activationfunctions['myfunc'] = (my_activation, my_activation_delta)

In [3]:
from binet.util import train
import time

x_tr, y_tr, x_va, y_va, x_te, y_te = load_dataset('mnist_bgimg', return_testset=True)
    
params = {'layersizes': (x_tr.shape[1], 1024, 1024, y_tr.shape[1]), 'max_iter':125, 
          'learning_rate':0.05, 'momentum': 0.8, 'verbose': True,
          'activation':'myfunc'}

net = NeuralNet( **params)
net = train(net, (x_tr, y_tr, x_va, y_va), skip_output=25)

 25:	Train-Error: 0.007736	Val-Error: 0.934976	Val-Score: 71.4500%	(16.22s)
 50:	Train-Error: 0.003295	Val-Error: 1.165723	Val-Score: 71.4000%	(31.05s)
 75:	Train-Error: 0.001304	Val-Error: 1.051073	Val-Score: 77.2500%	(45.92s)
100:	Train-Error: 0.000234	Val-Error: 1.118514	Val-Score: 78.8000%	(60.72s)


In [4]:
net.score(x_te, y_te)

0.78202

# System Information

In [5]:
print_system_information()

Host:                hawk: Linux-3.10.0-229.14.1.el7.x86_64-x86_64-with-centos-7.1.1503-Core
Date:                2015-10-12 10:44:29.097192
Python version:      3.4.3 (default, Mar  1 2015, 13:48:33) 
                     [GCC 4.8.2 20140120 (Red Hat 4.8.2-16)]
repository version:  commit abf08963b888becba55836a9a4e413f715dced4d

loaded modules:
	 IPython 4.0.0
	 binet 2015.10
	 matplotlib 1.4.3
	 numpy 1.9.2
	 pandas 0.16.0
	 scipy 0.16.0
	 sklearn 0.16.1
